In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import glob
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import models

In [3]:
PATH_TRAIN = "B:\\CV Drone Detection\\scaled\\train"
PATH_TEST = "B:\\CV Drone Detection\\scaled\\test"
PATH_VALID = "B:\\CV Drone Detection\\scaled\\valid"


transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),                  
    transforms.RandomVerticalFlip(),                       
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
    transforms.RandomRotation(20),                         
    transforms.RandomAffine(degrees=0, translate=(0.1,0.1)),
    transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
    transforms.Resize((128)),
    transforms.ToTensor(),
    transforms.RandomErasing(p=0.3, scale=(0.02, 0.2)),
    transforms.Normalize((0.5, 0.5, 0.5),
                         (0.5, 0.5, 0.5))
])


trainset = torchvision.datasets.ImageFolder(root=PATH_TRAIN, transform=transform)
trainloader = DataLoader(trainset, batch_size=4, shuffle=True, num_workers=2)

testset = torchvision.datasets.ImageFolder(root=PATH_TEST, transform=transform)
testloader = DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

classes = trainset.classes
print("Classes:", classes)

Classes: ['drone', 'notAdrone']


In [ ]:
model = models.resnet18(pretrained=True)

b:\CV Drone Detection\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
b:\CV Drone Detection\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\illya/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:04<00:00, 9.74MB/s]


In [8]:
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

In [12]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train(model, train_loader, valid_loader, criterion, optimizer, device, epochs=5):
    model.to(device)

    for epoch in range(epochs):
        model.train()
        train_loss = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)

        train_loss /= len(train_loader.dataset)

        model.eval()
        valid_loss = 0
        correct = 0

        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                valid_loss += loss.item() * images.size(0)
                preds = outputs.argmax(dim=1)
                correct += (preds == labels).sum().item()

        valid_loss /= len(valid_loader.dataset)
        accuracy = correct / len(valid_loader.dataset)

        print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, "
              f"Valid Loss: {valid_loss:.4f}, Accuracy: {accuracy:.4f}")


In [15]:
train(model, trainloader, testloader, loss_function, optimizer, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"), epochs=2)

KeyboardInterrupt: 